In [1]:
import glob
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler

# 폴더 경로 지정
folder_path = r'C:\Users\pc\Desktop\CNN\CIC-AndMal2017'

In [2]:
# 랜섬웨어 클래스별 파일에 있는 샘플 개수를 확인하기 위한 딕셔너리
samples_per_class = {}

# Ransomware 폴더 내의 10개 폴더 경로 리스트 받아오기
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')

# 모든 파일 로드 및 샘플 개수 확인
for folder in ransomware_folder_list:
    ransomware_class = folder.split('\\')[-1]  # 랜섬웨어 클래스명 추출

    file_paths = glob.glob(folder + '\\*.csv')
    total_samples = 0  # 클래스별 전체 샘플 개수 초기화
    for file_path in file_paths:
        data = pd.read_csv(file_path)
        total_samples += data.shape[0]  # 데이터프레임의 행 수 / 샘플 개수 더하기

    samples_per_class[ransomware_class] = total_samples

# 클래스별 샘플 개수 출력
for ransomware_class, num_samples in samples_per_class.items():
    print(f"{ransomware_class} 클래스의 샘플 개수: {num_samples}")

Charger 클래스의 샘플 개수: 39551
Jisut 클래스의 샘플 개수: 25672
Koler 클래스의 샘플 개수: 44555
Lockerpin 클래스의 샘플 개수: 25307
Pletor 클래스의 샘플 개수: 4715
PornDroid 클래스의 샘플 개수: 46082
RansomBO 클래스의 샘플 개수: 39859
Simplocker 클래스의 샘플 개수: 36340
SVpeng 클래스의 샘플 개수: 54161
WannaLocker 클래스의 샘플 개수: 32701


In [3]:
# 각 랜섬웨어 클래스별로 4,500개씩 무작위 샘플 추출 (비복원 추출)
num_samples_per_ransomware_class = 4500
all_ransomware_data = []

# Ransomware 폴더 내의 10개 폴더 경로 리스트 받아오기
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    ransomware_data = pd.DataFrame()

    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        ransomware_data = pd.concat([ransomware_data, data], ignore_index=True)

    # 랜섬웨어 클래스에서 4,500개 무작위 샘플 추출 (비복원 추출)
    if len(ransomware_data) > num_samples_per_ransomware_class:
        ransomware_subset = ransomware_data.sample(n=num_samples_per_ransomware_class, random_state=42, replace=False)
    else:
        ransomware_subset = ransomware_data.copy()
    all_ransomware_data.append(ransomware_subset)

# 모든 랜섬웨어 클래스의 데이터프레임들을 합치기
all_ransomware_data = pd.concat(all_ransomware_data, ignore_index=True)

print("특성추출 ransomware_subset의 샘플 개수:", len(all_ransomware_data))

특성추출 ransomware_subset의 샘플 개수: 45000


In [4]:
# 타깃 변수 열 이름 확인
target_variable = ' Label'  # 실제 타깃 변수 열 이름으로 수정

# X와 y로 데이터 분할
y = all_ransomware_data[target_variable]  # 타깃 변수
X = all_ransomware_data.drop([target_variable], axis=1)  # 타깃 변수 제외한 나머지 특성

# 불필요한 특성 제거 (예시: 'Flow ID', ' Timestamp', ' Source IP', ' Destination IP' 특성 제거)
unnecessary_features = ['Flow ID', ' Timestamp', ' Source IP', ' Destination IP']
X = X.drop(unnecessary_features, axis=1)

# 각 열(feature)에 Min-Max 스케일링 적용
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# chi2를 사용하여 특성 선택 (하위 36개의 특성 선택)
num_features_to_select = 36
selector = SelectKBest(score_func=chi2, k=num_features_to_select)
X_selected = selector.fit_transform(X_scaled, y)

# 선택된 특성들의 인덱스 추출
selected_feature_indices = selector.get_support(indices=True)

# 선택된 특성들의 컬럼명 추출
selected_feature_names = X.columns[selected_feature_indices]

# 최종 선택된 특성들의 데이터프레임 생성
X_final = X[selected_feature_names]

# 결과 출력
print("선택된 특성들의 컬럼명:")
print(selected_feature_names)
print("\n최종 선택된 특성들의 데이터프레임:")
print(X_final.head())

선택된 특성들의 컬럼명:
Index([' Source Port', ' Destination Port', ' Flow Duration',
       ' Total Fwd Packets', 'Total Length of Fwd Packets',
       ' Fwd Packet Length Max', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', ' Flow IAT Mean', ' Flow IAT Std',
       ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean',
       ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total',
       ' Bwd IAT Max', 'Fwd PSH Flags', 'FIN Flag Count', ' SYN Flag Count',
       ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count',
       ' Avg Bwd Segment Size', 'Subflow Fwd Packets', ' Subflow Fwd Bytes',
       'Init_Win_bytes_forward', ' Init_Win_bytes_backward',
       ' act_data_pkt_fwd', 'Active Mean', ' Active Std', ' Active Max',
       ' Active Min', ' Idle Std'],
      dtype='object')

최종 선택된 특성들의 데이터프레임:
    Source Port   Destination Port   Flow Duration   Total Fwd Packets  \
0         60774              40009          236683                   1   
1         

In [5]:
# 특성들의 가중치(Chi-square 통계량) 확인
chi2_scores = selector.scores_[selected_feature_indices]

# 특성들의 가중치를 기준으로 오름차순 정렬
sorted_indices = chi2_scores.argsort()
sorted_features = selected_feature_names[sorted_indices]
sorted_chi2_scores = chi2_scores[sorted_indices]

# 특성들의 가중치와 순위 출력
print("특성들의 순위와 가중치:")
for i, (feature, score) in enumerate(zip(sorted_features, sorted_chi2_scores), 1):
    print(f"{i}. {feature}: {score}")

특성들의 순위와 가중치:
1.  Flow IAT Std: 45.29844693371353
2.  Fwd IAT Std: 46.11572199853221
3.  Fwd Packet Length Max: 49.48327490132737
4. Bwd IAT Total: 52.35419129275548
5.  Bwd IAT Max: 53.28855112140505
6.  Bwd Packet Length Mean: 59.35788277916538
7.  Avg Bwd Segment Size: 59.35788277916539
8.  Active Min: 65.74880849826536
9.  Fwd IAT Mean: 68.96614988197021
10.  Idle Std: 72.00845782992076
11.  Bwd Packet Length Std: 72.72002073586562
12.  Flow IAT Mean: 76.7671529185573
13. Subflow Fwd Packets: 78.39417467495701
14.  Total Fwd Packets: 78.39417467495701
15.  Fwd IAT Min: 82.079150258402
16.  Flow IAT Max: 82.27561200165069
17.  Fwd IAT Max: 88.3433332410505
18.  Flow IAT Min: 89.5528542559586
19.  act_data_pkt_fwd: 91.63664916409286
20. Total Length of Fwd Packets: 107.3025452540851
21.  Subflow Fwd Bytes: 107.3025452540851
22.  ACK Flag Count: 108.26701030926633
23.  Init_Win_bytes_backward: 119.09366096064414
24. FIN Flag Count: 126.61111111109702
25. Active Mean: 132.6788756436581

In [6]:
# 각 랜섬웨어 클래스별로 4,000개씩 무작위 샘플 추출 (비복원 추출)
num_samples_per_ransomware_class = 4000
train_ransomware_data = []

# Ransomware 폴더 내의 10개 폴더 경로 리스트 받아오기
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    ransomware_data = pd.DataFrame()

    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        ransomware_data = pd.concat([ransomware_data, data], ignore_index=True)

    # 랜섬웨어 클래스에서 4,000개 무작위 샘플 추출 (비복원 추출)
    if len(ransomware_data) > num_samples_per_ransomware_class:
        ransomware_subset = ransomware_data.sample(n=num_samples_per_ransomware_class, random_state=42, replace=False)
    else:
        ransomware_subset = ransomware_data.copy()
    train_ransomware_data.append(ransomware_subset)

# 모든 랜섬웨어 클래스의 데이터프레임들을 합치기
train_ransomware_data_subset = pd.concat(train_ransomware_data, ignore_index=True)

print("train_ransomware_subset의 샘플 개수:", len(train_ransomware_data_subset))

train_ransomware_subset의 샘플 개수: 40000


In [7]:
from sklearn.preprocessing import MinMaxScaler

# 선택된 36가지 특성에 해당하는 열만 추출
selected_feature_columns = X_final.columns
selected_feature_values = train_ransomware_data_subset[selected_feature_columns]

# Min-Max 스케일링을 사용하여 선택된 특성들을 0과 1 사이의 값으로 정규화
scaler = MinMaxScaler()
selected_feature_values_normalized = scaler.fit_transform(selected_feature_values)

# 정규화된 특성들을 데이터프레임으로 변환
X_final_normalized = pd.DataFrame(selected_feature_values_normalized, columns=selected_feature_columns)

# 결과 출력
print("정규화된 특성들의 데이터프레임:")
print(X_final_normalized.head())

정규화된 특성들의 데이터프레임:
    Source Port   Destination Port   Flow Duration   Total Fwd Packets  \
0      0.927408           0.612217        0.001972            0.000000   
1      0.795654           0.001224        0.004341            0.000826   
2      0.532755           0.001224        0.001465            0.000551   
3      0.469076           0.000811        0.002538            0.000000   
4      0.264424           0.000811        0.005045            0.000000   

   Total Length of Fwd Packets   Fwd Packet Length Max  \
0                     0.000005                0.001085   
1                     0.000344                0.072021   
2                     0.000000                0.000000   
3                     0.000009                0.001924   
4                     0.000008                0.001677   

    Bwd Packet Length Mean   Bwd Packet Length Std   Flow IAT Mean  \
0                 0.067123                0.000000        0.001976   
1                 0.033048                0.0893

In [8]:
# 선택된 36가지 특성을 6x6 크기의 2차원 행렬로 변환
num_rows = 6
num_columns = 6
X_final_reshaped = X_final_normalized.values.reshape(-1, num_rows, num_columns)

# 결과 출력
print("6x6 크기의 2차원 행렬:")
print(X_final_reshaped[0])  # 첫 번째 샘플에 해당하는 6x6 행렬 출력

6x6 크기의 2차원 행렬:
[[9.27408402e-01 6.12217105e-01 1.97235440e-03 0.00000000e+00
  5.10477789e-06 1.08524073e-03]
 [6.71232877e-02 0.00000000e+00 1.97573890e-03 0.00000000e+00
  1.97573890e-03 1.97578056e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 6.71232877e-02 0.00000000e+00 5.10477789e-06
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [9]:
from PIL import Image
import os

# 이미지를 저장할 폴더를 생성합니다.
save_folder = r'C:\Users\pc\Desktop\CNN\Multiclass_Train_Image\\'

# 각 랜섬웨어 클래스 폴더를 생성합니다.
ransomware_classes = ["Charger", "Jisut", "Koler", "Lockerpin", "Pletor", "PornDroid", "RansomBO", "Simplocker", "SVpeng", "WannaLocker"]
for ransomware_class in ransomware_classes:
    os.makedirs(os.path.join(save_folder, ransomware_class), exist_ok=True)

# X_final_reshaped를 이미지로 변환하여 저장합니다.
num_samples = X_final_reshaped.shape[0]
for i in range(num_samples):
    # 1채널 16비트 PNG 이미지 생성
    image_data = X_final_reshaped[i]
    image_data = (image_data * 65535).astype('uint16')  # 16비트로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환

    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < 4000:
        label = "Charger"
    elif 4000 <= i < 8000:
        label = "Jisut"
    elif 8000 <= i < 12000:
        label = "Koler"
    elif 12000 <= i < 16000:
        label = "Lockerpin"
    elif 16000 <= i < 20000:
        label = "Pletor"
    elif 20000 <= i < 24000:
        label = "PornDroid"
    elif 24000 <= i < 28000:
        label = "RansomBO"
    elif 28000 <= i < 32000:
        label = "Simplocker"
    elif 32000 <= i < 36000:
        label = "SVpeng"
    else:
        label = "WannaLocker"

    # 이미지 저장
    image_filename = f"image_{i}.png"
    image.save(os.path.join(save_folder, label, image_filename))

print("이미지 변환 및 저장이 완료되었습니다.")

이미지 변환 및 저장이 완료되었습니다.


In [10]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 이미지 데이터를 저장한 폴더 경로
image_folder = r'C:\Users\pc\Desktop\CNN\Multiclass_Train_Image\\'

# 이미지 데이터를 불러오고 라벨을 지정합니다.
X_data = []
y_labels = []

label_names = ["Charger", "Jisut", "Koler", "Lockerpin", "Pletor", "PornDroid", "RansomBO", "Simplocker", "SVpeng", "WannaLocker"]


for i in range(num_samples):
    if i < 4000:
        label_index = 0           # Charger
    elif 4000 <= i < 8000:
        label_index = 1           # Jisut 
    elif 8000 <= i < 12000:
        label_index = 2           # Koler
    elif 12000 <= i < 16000:
        label_index = 3           # Lockerpin
    elif 16000 <= i < 20000:
        label_index = 4           # Pletor
    elif 20000 <= i < 24000:
        label_index = 5           # PornDroid
    elif 24000 <= i < 28000:
        label_index = 6           # RansomBO
    elif 28000 <= i < 32000:
        label_index = 7           # Simplocker
    elif 32000 <= i < 36000:
        label_index = 8           # SVpeng
    else:
        label_index = 9           # WannaLocker

    label_name = label_names[label_index]
    image_path = os.path.join(image_folder, label_name, f"image_{i}.png")
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    X_data.append(image)
    y_labels.append(label_index)

X_data = np.array(X_data)
y_labels = np.array(y_labels)

In [11]:
# Count the number of samples in each class
num_samples_per_class = []

for label_index in range(len(ransomware_classes)):
    num_samples = np.count_nonzero(y_labels == label_index)
    num_samples_per_class.append(num_samples)

print("Number of samples in each class:")
for i, ransomware_class in enumerate(ransomware_classes):
    print(f"{ransomware_class}: {num_samples_per_class[i]}")

Number of samples in each class:
Charger: 4000
Jisut: 4000
Koler: 4000
Lockerpin: 4000
Pletor: 4000
PornDroid: 4000
RansomBO: 4000
Simplocker: 4000
SVpeng: 4000
WannaLocker: 4000


In [12]:
# 새로 추출할 샘플 개수 설정
num_new_samples_per_ransomware_class = 500

# 샘플을 저장할 리스트
val_ransomware_subset = []

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    ransomware_data = pd.DataFrame()

    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        ransomware_data = pd.concat([ransomware_data, data], ignore_index=True)
    
    # 샘플에서 500개씩 추출
    if len(ransomware_data) > num_new_samples_per_ransomware_class:
        ransomware_subset = ransomware_data.sample(n=num_new_samples_per_ransomware_class, random_state=42, replace=False)
    else:
        ransomware_subset = ransomware_data.copy()
    val_ransomware_subset.append(ransomware_subset)

# 모든 랜섬웨어 클래스의 샘플 데이터프레임들을 합치기
val_ransomware_subset_subset = pd.concat(val_ransomware_subset, ignore_index=True)

print("Validation_ransomware_subset의 샘플 개수:", len(val_ransomware_subset_subset))


Validation_ransomware_subset의 샘플 개수: 5000


In [13]:
# 선택된 36가지 특성에 해당하는 열만 추출 (Validation 데이터셋)
val_selected_feature_values = val_ransomware_subset_subset[selected_feature_columns]

# Min-Max 스케일링을 사용하여 선택된 특성들을 0과 1 사이의 값으로 정규화 (Validation 데이터셋)
val_selected_feature_values_normalized = scaler.transform(val_selected_feature_values)

# 정규화된 특성들을 데이터프레임으로 변환 (Validation 데이터셋)
X_val_normalized = pd.DataFrame(val_selected_feature_values_normalized, columns=selected_feature_columns)

# 선택된 36가지 특성을 6x6 크기의 2차원 행렬로 변환 (Validation 데이터셋)
X_val_reshaped = X_val_normalized.values.reshape(-1, num_rows, num_columns)


In [14]:
# 이미지를 저장할 폴더를 생성합니다. (Validation 폴더)
save_folder_val = r'C:\Users\pc\Desktop\CNN\Multiclass_Validation_Image\\'

# 각 랜섬웨어 클래스 폴더를 생성합니다.
ransomware_classes = ["Charger", "Jisut", "Koler", "Lockerpin", "Pletor", "PornDroid", "RansomBO", "Simplocker", "SVpeng", "WannaLocker"]
for ransomware_class in ransomware_classes:
    os.makedirs(os.path.join(save_folder_val, ransomware_class), exist_ok=True)

# X_val_reshaped를 이미지로 변환하여 저장합니다.
num_samples_val = X_val_reshaped.shape[0]
for i in range(num_samples_val):
    # 1채널 16비트 PNG 이미지 생성
    image_data = X_val_reshaped[i]
    image_data = (image_data * 65535).astype('uint16')  # 16비트로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환

    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < 500:
        label = "Charger"
    elif 500 <= i < 1000:
        label = "Jisut"
    elif 1000 <= i < 1500:
        label = "Koler"
    elif 1500 <= i < 2000:
        label = "Lockerpin"
    elif 2000 <= i < 2500:
        label = "Pletor"
    elif 2500 <= i < 3000:
        label = "PornDroid"
    elif 3000 <= i < 3500:
        label = "RansomBO"
    elif 3500 <= i < 4000:
        label = "Simplocker"
    elif 4000 <= i < 4500:
        label = "SVpeng"
    else:
        label = "WannaLocker"

    # 이미지 저장
    image_filename = f"image_{i}.png"
    image.save(os.path.join(save_folder_val, label, image_filename))

print("Validation 이미지 변환 및 저장이 완료되었습니다.")


Validation 이미지 변환 및 저장이 완료되었습니다.


In [15]:
# 이미지 데이터를 저장한 폴더 경로 (Validation 폴더)
image_folder_val = r'C:\Users\pc\Desktop\CNN\Multiclass_Validation_Image\\'

# 이미지 데이터를 불러오고 라벨을 지정합니다. (Validation)
X_data_val = []
y_labels_val = []

label_names_val = ["Charger", "Jisut", "Koler", "Lockerpin", "Pletor", "PornDroid", "RansomBO", "Simplocker", "SVpeng", "WannaLocker"]

for i in range(num_samples_val):
    if i < 500:
        label_index = 0           # Charger
    elif 500 <= i < 1000:
        label_index = 1           # Jisut 
    elif 1000 <= i < 1500:
        label_index = 2           # Koler
    elif 1500 <= i < 2000:
        label_index = 3           # Lockerpin
    elif 2000 <= i < 2500:
        label_index = 4           # Pletor
    elif 2500 <= i < 3000:
        label_index = 5           # PornDroid
    elif 3000 <= i < 3500:
        label_index = 6           # RansomBO
    elif 3500 <= i < 4000:
        label_index = 7           # Simplocker
    elif 4000 <= i < 4500:
        label_index = 8           # SVpeng
    else:
        label_index = 9           # WannaLocker

    label_name = label_names_val[label_index]
    image_path = os.path.join(image_folder_val, label_name, f"image_{i}.png")
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    X_data_val.append(image)
    y_labels_val.append(label_index)

X_data_val = np.array(X_data_val)
y_labels_val = np.array(y_labels_val)

In [16]:
# Count the number of samples in each class
num_samples_per_class = []

# 라벨을 숫자로 변경하였으므로, 클래스의 인덱스를 기준으로 카운트합니다.
for label_index in range(len(ransomware_classes)):
    num_samples = np.count_nonzero(y_labels_val == label_index)
    num_samples_per_class.append(num_samples)

print("Number of samples in each class:")
for i, ransomware_class in enumerate(ransomware_classes):
    print(f"{ransomware_class}: {num_samples_per_class[i]}")

Number of samples in each class:
Charger: 500
Jisut: 500
Koler: 500
Lockerpin: 500
Pletor: 500
PornDroid: 500
RansomBO: 500
Simplocker: 500
SVpeng: 500
WannaLocker: 500


In [17]:
# 각 랜섬웨어 클래스별로 500개씩 무작위 샘플 추출 (비복원 추출)
num_samples_per_ransomware_class = 500
test_ransomware_data = []

# Ransomware 폴더 내의 10개 폴더 경로 리스트 받아오기
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    ransomware_data = pd.DataFrame()

    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        ransomware_data = pd.concat([ransomware_data, data], ignore_index=True)

    # 랜섬웨어 클래스에서 500개 무작위 샘플 추출 (비복원 추출)
    if len(ransomware_data) > num_samples_per_ransomware_class:
        ransomware_subset = ransomware_data.sample(n=num_samples_per_ransomware_class, random_state=42, replace=False)
    else:
        ransomware_subset = ransomware_data.copy()
    test_ransomware_data.append(ransomware_subset)

# 모든 랜섬웨어 클래스의 데이터프레임들을 합치기
test_ransomware_data = pd.concat(test_ransomware_data, ignore_index=True)

print("test_ransomware_subset의 샘플 개수:", len(test_ransomware_data))

test_ransomware_subset의 샘플 개수: 5000


In [18]:
# 선택된 36가지 특성에 해당하는 열만 추출 (Test 데이터셋)
test_selected_feature_values = test_ransomware_data[selected_feature_columns]

# Min-Max 스케일링을 사용하여 선택된 특성들을 0과 1 사이의 값으로 정규화 (Test 데이터셋)
test_selected_feature_values_normalized = scaler.transform(test_selected_feature_values)

# 정규화된 특성들을 데이터프레임으로 변환 (Test 데이터셋)
X_test_normalized = pd.DataFrame(test_selected_feature_values_normalized, columns=selected_feature_columns)

# 선택된 36가지 특성을 6x6 크기의 2차원 행렬로 변환 (Test 데이터셋)
X_test_reshaped = X_test_normalized.values.reshape(-1, num_rows, num_columns)

In [19]:
from PIL import Image
import os

# 이미지를 저장할 폴더를 생성합니다.
save_folder = r'C:\Users\pc\Desktop\CNN\Multiclass_Test_Image\\'

# 각 랜섬웨어 클래스 폴더를 생성합니다.
ransomware_classes = ["Charger", "Jisut", "Koler", "Lockerpin", "Pletor", "PornDroid", "RansomBO", "Simplocker", "SVpeng", "WannaLocker"]
for ransomware_class in ransomware_classes:
    os.makedirs(os.path.join(save_folder, ransomware_class), exist_ok=True)

# X_test_reshaped를 이미지로 변환하여 저장합니다.
num_samples = X_test_reshaped.shape[0]
for i in range(num_samples):
    # 1채널 16비트 PNG 이미지 생성
    image_data = X_test_reshaped[i]
    image_data = (image_data * 65535).astype('uint16')  # 16비트로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환

    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < 500:
        label = "Charger"
    elif 500 <= i < 1000:
        label = "Jisut"
    elif 1000 <= i < 1500:
        label = "Koler"
    elif 1500 <= i < 2000:
        label = "Lockerpin"
    elif 2000 <= i < 2500:
        label = "Pletor"
    elif 2500 <= i < 3000:
        label = "PornDroid"
    elif 3000 <= i < 3500:
        label = "RansomBO"
    elif 3500 <= i < 4000:
        label = "Simplocker"
    elif 4000 <= i < 4500:
        label = "SVpeng"
    else:
        label = "WannaLocker"

    # 이미지 저장
    image_filename = f"image_{i}.png"
    image.save(os.path.join(save_folder, label, image_filename))

print("이미지 변환 및 저장이 완료되었습니다.")

이미지 변환 및 저장이 완료되었습니다.


In [20]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# 이미지 데이터를 저장한 폴더 경로
image_folder = r'C:\Users\pc\Desktop\CNN\Multiclass_Test_Image\\'

# 이미지 데이터를 불러오고 라벨을 지정합니다.
X_data = []
y_labels = []

label_names = ["Charger", "Jisut", "Koler", "Lockerpin", "Pletor", "PornDroid", "RansomBO", "Simplocker", "SVpeng", "WannaLocker"]

for i in range(num_samples):
    if i < 500:
        label_index = 0           # Charger
    elif 500 <= i < 1000:
        label_index = 1           # Jisut 
    elif 1000 <= i < 1500:
        label_index = 2           # Koler
    elif 1500 <= i < 2000:
        label_index = 3           # Lockerpin
    elif 2000 <= i < 2500:
        label_index = 4           # Pletor
    elif 2500 <= i < 3000:
        label_index = 5           # PornDroid
    elif 3000 <= i < 3500:
        label_index = 6           # RansomBO
    elif 3500 <= i < 4000:
        label_index = 7           # Simplocker
    elif 4000 <= i < 4500:
        label_index = 8           # SVpeng
    else:
        label_index = 9           # WannaLocker

    label_name = label_names[label_index]
    image_path = os.path.join(image_folder, label_name, f"image_{i}.png")
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    X_data.append(image)
    y_labels.append(label_index)

X_data = np.array(X_data)
y_labels = np.array(y_labels)


In [21]:
# Count the number of samples in each class
num_samples_per_class = []

# 라벨을 숫자로 변경하였으므로, 클래스의 인덱스를 기준으로 카운트합니다.
for label_index in range(len(ransomware_classes)):
    num_samples = np.count_nonzero(y_labels == label_index)
    num_samples_per_class.append(num_samples)

print("Number of samples in each class:")
for i, ransomware_class in enumerate(ransomware_classes):
    print(f"{ransomware_class}: {num_samples_per_class[i]}")

Number of samples in each class:
Charger: 500
Jisut: 500
Koler: 500
Lockerpin: 500
Pletor: 500
PornDroid: 500
RansomBO: 500
Simplocker: 500
SVpeng: 500
WannaLocker: 500


In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 절반씩 줄여보기도 하기

def create_model(input_shape, num_classes):
    model = models.Sequential()

    # Convolution 1
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())

    # Maxpooling 1
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Convolution 2
    model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())

    # Maxpooling 2
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Convolution 3
    model.add(layers.Conv2D(256, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())

    # Maxpooling 3
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Flatten
    model.add(layers.Flatten())

    # FC 1
    model.add(layers.Dense(2048, activation='relu'))
    model.add(layers.Dropout(0.5))

    # FC 2
    model.add(layers.Dense(2048, activation='relu')) 
    model.add(layers.Dropout(0.5))

    # FC 3 (출력층)
    model.add(layers.Dense(num_classes, activation='softmax'))

    # 모델 컴파일
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'AUC'])
    
    return model

In [23]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 이미지 데이터 폴더 경로
train_data_folder = r'C:\Users\pc\Desktop\CNN\Multiclass_Train_Image'
val_data_folder = r'C:\Users\pc\Desktop\CNN\Multiclass_Validation_Image'
test_data_folder = r'C:\Users\pc\Desktop\CNN\Multiclass_Test_Image'

# 모델 생성
input_shape = (28, 28, 1)  # 입력 이미지의 크기와 채널 수
num_classes = 10  # 다중 분류 문제의 경우 클래스 개수는 10
model = create_model(input_shape, num_classes)

# ImageDataGenerator 생성
datagen = ImageDataGenerator(rescale=1./255)

# 학습 데이터 로드
train_generator = datagen.flow_from_directory(
    train_data_folder,
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    shuffle=True
)

# Validation 데이터 로드
val_generator = datagen.flow_from_directory(
    val_data_folder,
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='scategorical',
    batch_size=32,
    shuffle=False  # Validation 데이터는 섞지 않음
)

# Test 데이터 로드
test_generator = datagen.flow_from_directory(
    test_data_folder,
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
    shuffle=False  # Test 데이터는 섞지 않음
)

# 모델 학습
epochs = 50
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

# 모델 평가
test_loss, test_accuracy, test_auc = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
print("Test AUC:", test_auc)

Found 40000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
Epoch 1/50
1250/1250 [==============================] - 55s 43ms/step - loss: 2.2387 - accuracy: 0.1587 - auc: 0.5900 - val_loss: 2.1403 - val_accuracy: 0.1808 - val_auc: 0.6343
Epoch 2/50
1250/1250 [==============================] - 54s 43ms/step - loss: 2.1577 - accuracy: 0.1790 - auc: 0.6250 - val_loss: 2.1087 - val_accuracy: 0.1918 - val_auc: 0.6524
Epoch 3/50
1250/1250 [==============================] - 55s 44ms/step - loss: 2.1305 - accuracy: 0.1865 - auc: 0.6401 - val_loss: 2.0928 - val_accuracy: 0.1978 - val_auc: 0.6551
Epoch 4/50
1250/1250 [==============================] - 54s 43ms/step - loss: 2.1142 - accuracy: 0.1930 - auc: 0.6502 - val_loss: 2.0801 - val_accuracy: 0.2074 - val_auc: 0.6660
Epoch 5/50
1250/1250 [==============================] - 54s 43ms/step - loss: 2.1014 - accuracy: 0.1998 - auc: 0.6565 - val_loss: 2.0607 - val_accuracy: 0.207